In [ ]:
!nvidia-smi

!pip install transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install datasets
!pip install sacremoses
import sys, os, random
from tqdm import tqdm
from transformers.pipelines.text2text_generation import TranslationPipeline as trans_pipeline
import torch
from transformers.pipelines import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
target_lang = 'de' # 'de', 'hi'
print(target_lang)
trans_model_name = 'Helsinki-NLP/opus-mt-en-' + target_lang
backtrans_model_name = 'Helsinki-NLP/opus-mt-' + target_lang + '-en'

#device = xm.xla_device()
device = torch.device('cuda:0')

data_dir="drive/MyDrive/data/"
pos_file_name = "train_neg_full_preprocessed.txt"
pos_dest_file_name = "train_neg_full_translated.txt"

trans_tokenizer = AutoTokenizer.from_pretrained(trans_model_name)
trans_model = AutoModelForSeq2SeqLM.from_pretrained(trans_model_name)
translator = pipeline(task='translation', model=trans_model, tokenizer=trans_tokenizer, batch_size=32, device=device)

backtrans_tokenizer = AutoTokenizer.from_pretrained(backtrans_model_name)
backtrans_model = AutoModelForSeq2SeqLM.from_pretrained(backtrans_model_name)
backtranslator = pipeline(task='translation', model=backtrans_model, tokenizer=backtrans_tokenizer, batch_size=32, device=device)

trans_model.eval()
backtrans_model.eval()

s = ""

pos_file = open(data_dir+pos_file_name, encoding="utf8").read().split("\n")
for i in tqdm(range(0, len(pos_file), 32)):
  
  if random.random() < 0.5:
    continue

  ls = pos_file[i:i+32]
  for j in range(len(ls)):
    if len(ls[j])<513:
      pass
    else:
      t = ls[j][:512].split(" ")
      s_ = ""
      for k in range(len(t)-1):
        s_ += t[k]+" "
      ls[j] = s_[:-1]
  
  output = translator(ls, max_length=512)
  output = [obj['translation_text'] for obj in output]

  for j in range(len(output)):
    if len(output[j])<513:
      pass
    else:
     t = output[j][:512].split(" ")
     s_ = ""
     for k in range(len(t)-1):
       s_ += t[k]+" "
     output[j] = s_[:-1]
  
  output = backtranslator(output, max_length=512)
  for out in output:
    s+=out["translation_text"]+"\n"


with open(data_dir+pos_dest_file_name, 'w', encoding='UTF-8') as f:
        f.writelines(s)

